In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.calibration import calibration_curve
import torch 
import torch.nn as nn
import torch.nn.functional as F
import pickle
import yaml

import sys
sys.path.append("../")
from models import DiscreteNAM, NAM
from utils import discretize, get_dataset, get_bin_counts, get_discetized_run_data, get_run_data, get_ebm_run_data

sys.path.append("../run_scripts")
from epoch_functions import train_epoch_nam_pairs, test_epoch_nam_pairs

In [98]:
def get_preds_discrete_nam_(dataset, seed, split):
    
    # Read in model
    model = torch.load(f"../model_saves/discrete_nam_{dataset}_seed{seed}_split{split}.pt")
    
    # Read in args using yaml
    args_id = model.params_id
    with open(f"../run_parameters/discrete_nam_{dataset}_seed{seed}_split{split}_params{args_id}.yaml", "r") as f:
        args = yaml.safe_load(f)
    
    
    data_dict = \
        get_discetized_run_data(dataset, seed=seed, split=split, max_bins=args["max_bins"], use_feature_set=False)
    
    X_test_discrete = data_dict["X_test_discrete"]
    y_test = data_dict["y_test"]
    X_train_discrete = data_dict["X_train_discrete"]
    y_train = data_dict["y_train"]
    
    selected_pairs = model.pairs_list.cpu().numpy()
    X_test_interactions = X_test_discrete.values[:, selected_pairs]
    X_train_interactions = X_train_discrete.values[:, selected_pairs]

    batch_size = args["batch_size"]

    test_dataset = torch.utils.data.TensorDataset(
        torch.FloatTensor(X_test_discrete.values),
        torch.FloatTensor(X_test_interactions), 
        torch.FloatTensor(y_test),
    )
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.FloatTensor(X_train_discrete.values),
        torch.FloatTensor(X_train_interactions), 
        torch.FloatTensor(y_train),
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    _, test_preds = test_epoch_nam_pairs(model, test_loader, model_mains=model)
    _, train_preds = test_epoch_nam_pairs(model, train_loader, model_mains=model)
    
    return test_preds, y_test, train_preds, y_train

In [93]:
def get_preds_discrete_nam(dataset, seed):
    
    splits = [1, 2, 3, 4, 5]
    
    test_preds = 0
    train_aucs = []
    for split in splits:
        split_preds, y_test, split_train_preds, y_train = get_preds_discrete_nam_(dataset, seed, split)
        test_preds += split_preds
        
        split_train_preds = torch.sigmoid(split_train_preds).cpu().numpy()
        train_aucs.append(
            roc_auc_score(y_train, split_train_preds)
        )
        
    test_preds = test_preds / len(splits)
    
    return torch.sigmoid(test_preds).cpu().numpy(), y_test, train_aucs

In [43]:
def get_preds_nam_(dataset, seed, split, exu=False):
    
    # Read in model
    model = torch.load(f"../model_saves/nam_exu{exu}_{dataset}_seed{seed}_split{split}.pt")
    
    # Read in args using yaml
    args_id = model.params_id
    with open(f"../run_parameters/nam_exu{exu}_{dataset}_seed{seed}_split{split}_params{args_id}.yaml", "r") as f:
        args = yaml.safe_load(f)
        
    data_dict = get_run_data(dataset, seed=seed, split=split, preprocess=True)
    X_test = data_dict["X_test"]
    y_test = data_dict["y_test"]
        
    selected_pairs = model.pairs_list.cpu().numpy()
    X_test_interactions = X_test.values[:, selected_pairs]

    batch_size = args["batch_size"]

    test_dataset = torch.utils.data.TensorDataset(
        torch.FloatTensor(X_test.values),
        torch.FloatTensor(X_test_interactions), 
        torch.FloatTensor(y_test),
    )
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    _, preds = test_epoch_nam_pairs(model, test_loader, model_mains=model)
    return preds, y_test

In [60]:
def get_preds_nam(dataset, seed, exu=False):
    
    splits = [1, 2, 3, 4, 5]
    
    test_preds = 0
    train_preds = 0
    for split in splits:
        split_preds, y_test, split_train_preds, y_train = get_preds_nam_(dataset, seed, split, exu=exu)
        test_preds += split_preds
        train_preds += split_train_preds
        
    preds = preds / len(splits)
    return torch.sigmoid(preds).cpu().numpy(), y_test

In [6]:
def get_preds_ebm(dataset, seed):
    
    # Read in model
    with open(f"../model_saves/ebm_{dataset}_seed{seed}.pkl", "rb") as f:
        ebm = pickle.load(f)
    
    # Split doesn't matter because test data is the same
    data_dict = get_ebm_run_data(dataset, seed=seed)
    X_test = data_dict["X_test"]
    
    preds = ebm.predict_proba(X_test)[:, 1]
    return preds, data_dict["y_test"]

In [7]:
def get_preds_xgboost(dataset, seed):
    
    # Read in model
    with open(f"../model_saves/xgboost_{dataset}_seed{seed}.pkl", "rb") as f:
        xgboost = pickle.load(f)
    
    # Split doesn't matter because test data is the same
    data_dict = get_ebm_run_data(dataset, seed=seed)
    X_test = data_dict["X_test"]
    
    preds = xgboost.predict_proba(X_test)[:, 1]
    return preds, data_dict["y_test"]

In [99]:
# datasets = ["higgs", "phoneme", "miniboone", "housing", "adult", "churn", "spam"]
datasets = ["adult"]
# seeds = [10, 11, 12, 13, 14]
seeds = [10, 11]
splits = [1, 2, 3, 4, 5]
# splits = [1]
models = ["Discrete_NAM"]
# models = ["NAM", "NAM_ExU"]

results = []
for dataset in datasets:
    print("DATASET", dataset)
    for model in models:
        for seed in seeds:
            if model == "EBM":
                preds, y_test = get_preds_ebm(dataset, seed)
            elif model == "XGBoost":
                preds, y_test = get_preds_xgboost(dataset, seed)
            elif model == "Discrete_NAM":
                preds, y_test, train_aucs = get_preds_discrete_nam(dataset, seed)
            elif model == "NAM":
                preds, y_test = get_preds_nam(dataset, seed)
            elif model == "NAM_ExU":
                preds, y_test = get_preds_nam(dataset, seed, exu=True)
            
            else:
                
                continue
            
            auc = roc_auc_score(y_test, preds)
            results.append([
                dataset, model, seed, auc, "TEST"
            ])
            
            results.append([
                dataset, model, seed, np.mean(train_aucs), "TRAIN"
            ])
            
results_df = pd.DataFrame(results, columns=["dataset", "model", "seed", "auc", "set"])
results_df

DATASET adult


,dataset,model,seed,auc,set
0,adult,Discrete_NAM,10,0.923233,TEST
1,adult,Discrete_NAM,10,0.933705,TRAIN
2,adult,Discrete_NAM,11,0.924933,TEST
3,adult,Discrete_NAM,11,0.934379,TRAIN


In [97]:
results_df

,dataset,model,seed,auc,set
0,adult,Discrete_NAM,10,0.923005,TEST
1,adult,Discrete_NAM,10,0.935866,TRAIN
2,adult,Discrete_NAM,11,0.924648,TEST
3,adult,Discrete_NAM,11,0.934553,TRAIN


In [91]:
results_df.groupby(["dataset", "model", "set"])["auc"].agg(["mean", "std"])

mean       std
dataset model        set                      
adult   Discrete_NAM TEST   0.919259  0.008890
                     TRAIN  0.921510  0.013499

In [87]:
results_df.groupby(["dataset", "model", "set"])["auc"].agg(["mean", "std"])

mean       std
dataset model        set                      
adult   Discrete_NAM TEST   0.926002  0.003043
                     TRAIN  0.933301  0.001000

In [38]:
results_df.groupby(["dataset", "model"])["auc"].agg(["mean", "std"])

,,mean,std
dataset,model,,
adult,Discrete_NAM,0.926002,0.003043


In [14]:
results_df = results_df[results_df["dataset"] != "phoneme"]

In [12]:
# Get dataframe with models as columns and datasets as rows
means_df = results_df.groupby(["dataset", "model"])["auc"].mean().reset_index()
means_df = means_df.pivot(index="dataset", columns="model", values="auc")

# Change the order of the columns
col_order = ["XGBoost", "EBM", "NAM", "NAM_ExU", "Discrete_NAM"]
means_df = means_df[col_order]

stds_df = results_df.groupby(["dataset", "model"])["auc"].std().reset_index()
stds_df = stds_df.pivot(index="dataset", columns="model", values="auc")
stds_df = stds_df[col_order]

means_df = means_df.round(3).map("{:.3f}".format)
stds_df = stds_df.round(3).map("{:.3f}".format)

# Convert to string and add plus/minus in between
means_and_str_df = means_df.astype(str) + " ± " + stds_df.astype(str)
print(means_and_str_df.to_latex())

\begin{tabular}{llllll}
\toprule
model & XGBoost & EBM & NAM & NAM_ExU & Discrete_NAM \\
dataset &  &  &  &  &  \\
\midrule
adult & 0.925 ± 0.003 & 0.923 ± 0.003 & 0.913 ± 0.003 & 0.914 ± 0.003 & 0.926 ± 0.003 \\
churn & 0.831 ± 0.005 & 0.856 ± 0.010 & 0.854 ± 0.010 & 0.851 ± 0.011 & 0.857 ± 0.009 \\
higgs & 0.801 ± 0.003 & 0.797 ± 0.002 & 0.801 ± 0.002 & 0.800 ± 0.002 & 0.793 ± 0.001 \\
housing & 0.968 ± 0.001 & 0.962 ± 0.001 & 0.950 ± 0.002 & 0.950 ± 0.003 & 0.961 ± 0.001 \\
miniboone & 0.984 ± 0.000 & 0.978 ± 0.001 & 0.972 ± 0.001 & 0.969 ± 0.007 & 0.978 ± 0.001 \\
phoneme & 0.954 ± 0.006 & 0.937 ± 0.008 & 0.918 ± 0.008 & 0.918 ± 0.008 & 0.931 ± 0.009 \\
spam & 0.990 ± 0.003 & 0.987 ± 0.004 & 0.983 ± 0.005 & 0.983 ± 0.004 & 0.985 ± 0.006 \\
\bottomrule
\end{tabular}



In [15]:


# Get dataframe with dataset as columns and model as rows, with each entry as mean +- std
means_df = results_df.groupby(["dataset", "model"])["auc"].mean().reset_index()
means_df = means_df.pivot_table(index="model", columns="dataset", values="auc")

stds_df = results_df.groupby(["dataset", "model"])["auc"].std().reset_index()
stds_df = stds_df.pivot_table(index="model", columns="dataset", values="auc")

means_df = means_df.round(3).map("{:.3f}".format)
stds_df = stds_df.round(3).map("{:.3f}".format)

# Convert to string and add plus/minus in between
means_and_str_df = means_df.astype(str) + " ± " + stds_df.astype(str)
print(means_and_str_df.to_latex())

\begin{tabular}{lllllll}
\toprule
dataset & adult & churn & higgs & housing & miniboone & spam \\
model &  &  &  &  &  &  \\
\midrule
Discrete_NAM & 0.926 ± 0.003 & 0.857 ± 0.009 & 0.793 ± 0.001 & 0.961 ± 0.001 & 0.978 ± 0.001 & 0.985 ± 0.006 \\
EBM & 0.923 ± 0.003 & 0.856 ± 0.010 & 0.797 ± 0.002 & 0.962 ± 0.001 & 0.978 ± 0.001 & 0.987 ± 0.004 \\
XGBoost & 0.925 ± 0.003 & 0.831 ± 0.005 & 0.801 ± 0.003 & 0.968 ± 0.001 & 0.984 ± 0.000 & 0.990 ± 0.003 \\
\bottomrule
\end{tabular}

